#  Patient Admission Scheduling

## Problem Formulation


Given a hospital with multiple departments, each containing rooms and beds, the challenge is to optimize the management of patients and bed availability. The goal is to efficiently assign a bed for each night a patient needs to stay in the hospital, considering the constraints of allocating beds to rooms and rooms to departments.

## Project Goal

The goal of the project is to develop an intelligent agent, documenting the design and implementation steps.

### Team Members
- 18851 Luís Gonçalves Oliveira
- 16443 Francisco Moreira Rebelo
- 21990 Allan Sales Aleluia


## Agent

### PEAS - Agent Structure

PEAS is a conceptual framework used in Artificial Intelligence to describe an agent system. The acronym PEAS stands for:

- **Performance Measure:** Refers to criteria that define the success of the agent in a specific task. These measures are used to evaluate how well the agent is performing its actions in the environment.

- **Environment:** Represents the context or scenario in which the agent operates. It may include all elements that affect or are affected by the agent, such as physical objects, other agents, external conditions, etc.

- **Actuators:** Are the means by which the agent brings about changes in the environment. These can be motors, communication devices, or any other means by which the agent can influence its surroundings.

- **Sensors:** Are the means by which the agent perceives its environment. Sensors provide information to the agent about the current state of the environment, enabling the agent to make informed decisions.

### Example of PEAS Application:

| **AGENT TYPE**      | **Performance Measure**                      | **Environment**                          | **Actuators**           | **Sensors** |
|:-------------------:|:-------------------------------------------:|:---------------------------------------:|:-----------------------:|:------------:|
| Hospital Scheduling | Efficient bed distribution; Minimize scheduling errors | Hospital area, including wards and surgery rooms | Automated scheduling system in Python | Bed occupancy sensors, information about available doctors and nurses |


### Task Environment for Hospital Scheduling

The task environment for the hospital scheduling agent is characterized as follows:

- **Fully-observable:** The agent has complete visibility into the current state of the hospital, including bed occupancy, availability of medical staff, and scheduling information.

- **Single-agent:** The scheduling decisions are made by a single agent, specifically the hospital scheduling system.

- **Deterministic:** The outcome of scheduling actions is predictable; scheduling decisions directly influence the subsequent state of bed occupancy and medical staff availability.

- **Sequential:** Scheduling decisions made in the current state affect future states; for example, assigning a bed and schedule to one patient may impact the availability for another patient.

- **Dynamic:** The environment is dynamic, as bed occupancy and medical staff availability can change over time due to patient admissions, discharges, and staff schedules.

- **Discrete:** The scheduling system operates within a discrete framework, with a finite number of beds, medical staff, and scheduling slots.

- **Known:** The outcomes of scheduling actions are known to the system; the impact of assigning a bed or schedule is understood.

This task environment outlines the key characteristics and dynamics relevant to the hospital scheduling agent's decision-making process.



### AIMA Code Files Used

The team incorporated the following files from the AIMA code:

- `csp.py`
- `utils.py`
- `search.py`

In [ ]:
from csp import *
from utils import *
from search import *

## Variables

- **\(Beds\):** Set of available beds in the hospital.
- **\(Patients\):** Set of patients requiring scheduling.


## Domains

So, for each patient, the domain is initially set to be all available beds (1 through 8). As the code progresses and constraints are applied, certain beds are eliminated from the domain based on the patient's department, telemetry requirements, and oxygen requirements.

For example, when the code iterates through patients and room information to apply constraints, it selectively removes beds from the patient's domain based on compatibility with the room's properties and the patient's requirements. The goal is to narrow down the possible bed assignments for each patient, and the final solution is found using the constraint satisfaction solver.


## Input Data

We will use the file provided by the professor 'P01 Test Data' and we will input the data in the input file named input.py

In [ ]:
def parse_dataset(dataset):
    departments = []
    rooms = {}
    beds = {}
    patients = {}

    lines = dataset.strip().split('\n')
    current_section = None

    for line in lines:
        words = line.split()

        if not words:
            continue

        if words[0] in ['DEPARTMENTS:', 'ROOMS:', 'BEDS:', 'PATIENTS:']:
            current_section = words[0].rstrip(':')
            continue

        if current_section == 'DEPARTMENTS':
            if words[0] == 'END.':
                break

            department_id, department_name = words
            departments.append((int(department_id), department_name))

        elif current_section == 'ROOMS':
            if words[0] == 'END.':
                break

            room_id = words[0]
            room_name = ' '.join(words[1:-3])  # Unir os elementos restantes como nome da sala
            room_capacity = words[-3]
            room_department = words[-1]

            rooms[room_id] = {'name': room_name, 'capacity': int(room_capacity), 'department': int(room_department)}

        elif current_section == 'BEDS':
            if words[0] == 'END.':
                break

            bed_id, bed_room_id = words
            beds[bed_id] = {'room': bed_room_id}

        elif current_section == 'PATIENTS':
            if words[0] == 'END.':
                break

            patient_id, patient_name, patient_age, patient_gender, patient_admission_day, patient_discharge_day = words
            patients[int(patient_id)] = {
                'name': patient_name,
                'age': int(patient_age),
                'gender': patient_gender,
                'admission_day': int(patient_admission_day),
                'discharge_day': int(patient_discharge_day)
            }

    return departments, rooms, beds, patients

## Constraints

To make sure the schedule we create is practical and works well in real-life situations, we've set up some rules for the algorithm to follow. These rules help solve the hospital scheduling problem effectively by considering various factors. Here are the constraints we've established:

**1. Patient-Bed Assignment Constraint:**
   - *Goal:* Protect patient privacy and prevent infections.
   - *How It Works:* No two patients can share the same bed on the same night.

**2. Room Gender Limitation Constraint:**
   - *Goal:* Ensure patient comfort, privacy, and safety.
   - *How It Works:* Rooms may have specific gender requirements, like being for only females, only males, the same gender in the same night, or no gender restrictions.

**3. Equipment-Specific Room Assignment Constraint:**
   - *Goal:* Match rooms with the medical equipment a patient needs.
   - *How It Works:* Assigning rooms based on the equipment necessary for a patient's care.

**4. Departmental Specialization Constraint:**
   - *Goal:* Improve patient care by using specialized departments.
   - *How It Works:* Patients are assigned to departments that specialize in treating their specific medical condition.

**5. Departmental Age Limit Constraint:**
   - *Goal:* Ensure patients are in departments suitable for their age.
   - *How It Works:* Departments can have age restrictions to make sure patients are in the right medical environment for their age group.

**6. Additional Constraint: Patient Acuity Level Constraint:**
   - *Goal:* Customize patient assignments based on care intensity needs.
   - *How It Works:* Patients with higher care needs are placed in areas with closer monitoring, while those needing less attention go to standard care areas.

These rules guide the algorithm to create schedules that consider patient needs, privacy, safety, and specialized medical care, making sure the hospital operates efficiently and effectively.

## Results

## Repository 

https://github.com/a18851/IA_G09/

## References

https://saiconference.com/IntelliSys

http://aima.cs.berkeley.edu/

https://github.com/aimacode

